In [1]:
path2Data="cities/"

In [2]:
import glob
import math

In [88]:
# reads the POS tags of the tweets in path2Data
# Since numbers and such give many unique ''words'' we can specify which to replace
# currently replacing '#'-hashtags, '@'-usernames, 'U'-url links, 'E'-emoticons, '$'- numeral , ','-punctuations, 'G'-unknown tag
# we are removing some formating symbols eg. ":" which is tagged to '~'
replaceables = ['#', '@', 'U', 'E', '$', ',', 'G']
#replaceables = []
removables = ['~']
def cleanTweet(tweet, tweet_pos):
    tweet_l = tweet.split()
    tweet_pos_l = tweet_pos.split()

    if len(tweet_l) != len(tweet_pos_l):
        for i, item in enumerate(tweet_l):
            print (tweet_l[i], ',' , tweet_pos_l[i])
        
    clean_tweet = []
    for i, item in enumerate(tweet_l):
        #print (item)
        #print (tweet_pos_l[i])
        if tweet_pos_l[i] in replaceables:
            clean_tweet.append(tweet_pos_l[i])
        elif tweet_pos_l[i] in removables:
            None
        else:
            clean_tweet.append(item.lower())
    
    #print (clean_tweet)
    return clean_tweet

In [89]:
filenum = 0
Vcount=0
Vinv={}          # index to word map
V={}             # word to index
idf={}           # forwardIndex
tf={}            # (word, numberOfWords)
locRates={}      # HIV rates based on locations
#inverseIndex={}  
N = 0
for file in glob.glob(path2Data+'*.tsv'):
    filenum = filenum + 1  #serves as an index for the file name
    prefix = file.split('.')[0]
    locRates[filenum] = int(prefix.split('_')[-1])
    
    lines = []
    with open(file, 'r') as f:
        lines = f.readlines()
    #DEBUG
    #print (file + " file num: " + str(filenum) + " num tweets: " + str(len(lines)) )
    
    unique_words = set([])
    for line in lines:
        ll = line.split('\t')
        tweet = ll[0].strip()
        tweet_pos = ll[1].strip()
        
        for word in cleanTweet(tweet, tweet_pos):
            if word not in V:
                V[word]= Vcount
                Vinv[Vcount]=word
                Vcount = Vcount + 1
            
            if V[word] not in idf:
                idf[ V[word] ] = []
            
            if filenum not in tf:
                tf[filenum] = {}
            
            freq = tf[filenum].get(V[word], 0)
            tf[filenum][ V[word] ] = freq + 1
            
            if word not in unique_words:
                idf[ V[word] ].append(filenum)
                unique_words.add(word)
            
N = filenum
VocabSize=len(V.keys())


In [90]:
print ("vocabSize: ", VocabSize)
print ("docSize: ", N)
print ("labels: ", len(locRates))

vocabSize:  184183
docSize:  1504
labels:  1504


In [91]:
from statistics import *
mu = mean(locRates.values())
print ("mean: ",  mu )
med = median(locRates.values())
print ("median: ",  median(locRates.values()) )
print ("max: ", max(locRates.values()) )
print ("min: ", min(locRates.values()) )
#print ( median_low(locRates.values()) )
#print ( median_high(locRates.values()) )
sigma = stdev(locRates.values())
print ("standard diviation: ", sigma )

mean:  133.54255319148936
median:  82.5
max:  2084
min:  11
standard diviation:  160.6118525618946


In [92]:
# Version 2: generating Valid Keys for corresponding to the top/bottom HIV rates
#sample size is 2 (p*N), p*N for lower and p*N for upper
def sampleItems(locRates, p):
    items = []
    for key in locRates:
        items.append((key, locRates[key]))

    sorted_locRates = sorted(items, key=lambda student: student[1]) 
    total_items = len(items)
    sampleSize = (int) (p*total_items)
    print ("sample size: ", sampleSize)
    
    ret = []
    lab = {}
    for i, item in enumerate(sorted_locRates):
        if i < sampleSize:
            ret.append(item[0])
            lab[item[1]] = 0
            
        if i >= total_items - sampleSize:
            ret.append(item[0])
            #lab.append(1)
            lab[item[1]] = 1
    
    print ("samped items size: ", len(ret))
    return (ret,lab)
    

In [110]:
print (len(locRates)*.1)
(validKeys, labels) = sampleItems(locRates, .1)
print (len(validKeys), len(labels.keys()))
#print (validKeys)

150.4
sample size:  150
samped items size:  300
300 148


In [64]:
# Version 1: generating Valid Keys for corresponding to the top/bottom HIV rates Q1 and Q3 standardize
# using IQR
keys = []
newrates = []
for key in locRates:
    keys.append(key)
    newrates.append( (locRates[key] - mu)/sigma )
    
q1 = stdev(newrates) * -0.67 + mean(newrates)
q3 = stdev(newrates) *  0.67 + mean(newrates)

validKeys = []
labels = []
eps = 0.1
highCount = 0
lowCount = 0
for i, rate in enumerate(newrates):
    if rate < q1+eps:
        #lowRates.append(rate)
        validKeys.append( keys[i] ) 
        labels.append(0)
        lowCount = lowCount + 1
    if rate > q3-eps:
        #highRates.append(rate)
        validKeys.append( keys[i] ) 
        labels.append(1)
        highCount = highCount + 1

In [68]:
print ( "sample size: ", len(validKeys))
print ( "percentage: ", len(validKeys)/len(newrates)*100 )
print ( "number of Cities with High HIV rates: {0} ({2:.3f}) Low HIV rates: {1} ({3:.3f})".format(highCount, lowCount, highCount/len(validKeys), lowCount/len(validKeys)) )

sample size:  448
percentage:  29.78723404255319
number of Cities with High HIV rates: 219 (0.489) Low HIV rates: 229 (0.511)


In [94]:
def tfidf(docID, wordID, tf, idf):
    tf_0 = 0.5 +  tf[docID].get(wordID, 0)
    idf_0 = math.log( 1 + N/len(idf[wordID]))
    return (tf_0 * idf_0)

In [95]:
from sklearn import linear_model
from scipy.sparse import csr_matrix
import numpy as np
import random
random.seed(1000)


In [26]:
#Xtrain = csr_matrix ( (len(trainIndices) , VocabSize), dtype=float )
#Xtest = csr_matrix (  (len(testIndices) , VocabSize), dtype=float )
Ytrain = []
Ytest = []
#New generate matricies
row = []
col = []
data = []
for i, docID in enumerate(validKeys):
    for wordID in tf[docID]:
        row.append(i)
        col.append(wordID)
        data.append(tfidf(docID, wordID, tf,idf) ) 
        
Ytrain = labels
Xtrain = csr_matrix ( (np.array(data),(np.array(row),np.array(col))), shape=(len(validKeys),VocabSize), dtype=float)


In [111]:
#sample data
p = 1.00 #percentage of sampled data in
Ytrain = []
Ytest = []
n = len(validKeys)
validKeys0 = validKeys[:(int)(n/2)]
validKeys1 = validKeys[(int)(n/2):]

#need to keep it balanced
dataSize = len(validKeys0)
trainIndices0 = random.sample ( validKeys0, (int) (p*(dataSize)) ) #indicies start at 0
testIndices0 = set(validKeys0) - set(trainIndices0)

dataSize = len(validKeys1)
trainIndices1 = random.sample ( validKeys1, (int) (p*(dataSize)) ) #indicies start at 1
testIndices1 = set(validKeys1) - set(trainIndices1)

testIndices = testIndices0.union(testIndices1)
trainIndices = trainIndices0 + trainIndices1

#Old generate matricies
row = []
col = []
data = []
for i, docID in enumerate(trainIndices):
    for wordID in tf[docID]:
        row.append(i)
        col.append(wordID)
        data.append(tfidf(docID, wordID, tf,idf) ) 
    # uncomment to use regression
    # Ytrain.append(locRates[docID]) 
    # used for classification
    Ytrain.append (labels[ locRates[docID] ])
Xtrain = csr_matrix ( (np.array(data),(np.array(row),np.array(col))), shape=(len(trainIndices),VocabSize), dtype=float)

In [96]:
#generate matricies (testing Data) Not used since we are doing cross-validation instead.
row = []
col = []
data = []
for i, docID in enumerate(testIndices):
    for wordID in tf[docID]:
        row.append(i)
        col.append(wordID)
        data.append(tfidf(docID, wordID, tf,idf) ) 
        # Xtrain[i, wordID] = tfidf(docID, wordID, tf,idf)
    #Ytest.append(locRates[docID])
    Ytest.append (labels[ locRates[docID] ])
Xtest = csr_matrix ( (np.array(data),(np.array(row),np.array(col))), shape=(len(testIndices),VocabSize), dtype=float)

In [112]:
print (sum(Ytrain), sum(Ytest))
print (Xtrain.shape, Xtest.shape)

150 0
(300, 184183) (60, 184183)


In [98]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn import cross_validation
clf = MultinomialNB()
Y_hat = clf.fit(Xtrain, Ytrain).predict(Xtest)
cm = conf_mat(Y_hat, Ytest) 
print ("confusion matrix:")
print ("                             true label highHIVRates, true label lowHIVRates  ")
print ("predicted label highHIVRates:  {0:{width}d} {1:{width}d}".format(cm[0], cm[1],width=15))
print ("predicted label lowHIVRates :  {0:{width}d} {1:{width}d}".format(cm[3], cm[2],width=15))
print ("[fold {0}], accuracy: {1:.5f}, precision: {2:.5f}, recall: {3:.5f}".
      format(k, (cm[0]+cm[2])/(len(Y_hat)), cm[0]/(cm[0]+cm[1]), cm[0]/(cm[0]+cm[3])  ) )

#acc = (cm[0]+cm[2])/(len(Y_hat)) + acc
#prec = prec + cm[0]/(cm[0]+cm[1])
#recal = recal + cm[0]/(cm[0]+cm[3])
print ("")


23 18 12 7
confusion matrix:
                             true label highHIVRates, true label lowHIVRates  
predicted label highHIVRates:               23              18
predicted label lowHIVRates :                7              12
[fold 0], accuracy: 0.58333, precision: 0.56098, recall: 0.76667



In [84]:
print (Ytest)
print (Y_hat)

[1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]


In [37]:
# standardization? needed for training?
ss = [0]*VocabSize
Mu = Xtrain.sum(0)/N #sum over all rows
#print (Mu.shape)
#print (VocabSize)
count = 0
for i,j in zip(row, col):
    ss[j] = ss[j] + ((data[count] - Mu[0,j]) ** 2)
    count = count + 1

#print (len(data))
#print(Mu[0,:])
Sigma = []
for item in ss:
    Sigma.append( math.sqrt(item/N) )

xvals = []
count = 0
for i,j in zip(row,col):
    xvals.append((data[count] - Mu[0, j])/Sigma[j])
    count = count + 1

X_s = csr_matrix ( (np.array(xvals),(np.array(row),np.array(col))), shape=(len(trainIndices),VocabSize), dtype=float)

(1, 184183)
184183


In [113]:
def conf_mat(Y_hat, Y):
    tp = fp = tn = fn = 0
    for i,j in zip(Y_hat, Y):
        if i == 1:
            if i == j:
                tp = tp + 1
            else:
                fp = fp + 1
        elif i == 0:
            if i == j:
                tn = tn + 1
            else: 
                fn = fn + 1
        else:
            print (" j should only be 0 or 1, however", j , "was encountered.")
    print (tp, fp, tn, fn)
    return [tp, fp, tn, fn]
    
    

In [114]:
#using naive Bayes for Classification
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn import cross_validation
#from operator import add;
clf = MultinomialNB()
y = np.array(Ytrain)
K = 5
acc = 0
prec = 0
recal = 0
k_fold = cross_validation.StratifiedKFold(Ytrain, n_folds=K,shuffle=True)

for k, (train, test) in enumerate(k_fold):
    Y_hat = clf.fit(Xtrain[train], y[train]).predict(Xtrain[test])
    cm = conf_mat(Y_hat, y[test]) 
    print ("confusion matrix:")
    print ("                             true label highHIVRates, true label lowHIVRates  ")
    print ("predicted label highHIVRates:  {0:{width}d} {1:{width}d}".format(cm[0], cm[1],width=15))
    print ("predicted label lowHIVRates :  {0:{width}d} {1:{width}d}".format(cm[3], cm[2],width=15))
    print ("[fold {0}], accuracy: {1:.5f}, precision: {2:.5f}, recall: {3:.5f}".
          format(k, (cm[0]+cm[2])/(len(Y_hat)), cm[0]/(cm[0]+cm[1]), cm[0]/(cm[0]+cm[3])  ) )
    acc = (cm[0]+cm[2])/(len(Y_hat)) + acc
    prec = prec + cm[0]/(cm[0]+cm[1])
    recal = recal + cm[0]/(cm[0]+cm[3])
    print ("")

print ("average acc: {0:.5f}, average precision: {1:.5f}, average recall: {2:.5f}".format(acc/K, prec/K, recal/K))

22 15 15 8
confusion matrix:
                             true label highHIVRates, true label lowHIVRates  
predicted label highHIVRates:               22              15
predicted label lowHIVRates :                8              15
[fold 0], accuracy: 0.61667, precision: 0.59459, recall: 0.73333

20 21 9 10
confusion matrix:
                             true label highHIVRates, true label lowHIVRates  
predicted label highHIVRates:               20              21
predicted label lowHIVRates :               10               9
[fold 1], accuracy: 0.48333, precision: 0.48780, recall: 0.66667

23 11 19 7
confusion matrix:
                             true label highHIVRates, true label lowHIVRates  
predicted label highHIVRates:               23              11
predicted label lowHIVRates :                7              19
[fold 2], accuracy: 0.70000, precision: 0.67647, recall: 0.76667

21 14 16 9
confusion matrix:
                             true label highHIVRates, true label lowHIV

In [41]:
print ((int) (len(Ytrain)/2))

150


In [ ]:
clf = linear_model.LinearRegression()
clf.fit (Xtrain, Ytrain) 
clf.coef_

In [80]:
#1.0 is best possible score, lower is worse
clf.score(Xtest, Ytest)

0.092807781717191729

In [82]:
clf = linear_model.Ridge()
clf.fit (Xtrain, Ytrain) 
clf.coef_
#1.0 is best possible score, lower is worse
clf.score(Xtest, Ytest)

0.12343089742761792

In [ ]:
clf = linear_model.ElasticNet()
clf.fit (Xtrain, Ytrain) 
clf.coef_
#1.0 is best possible score, lower is worse
clf.score(Xtest, Ytest)

In [60]:

#Debug
lines = []
with open( path2Data + 'Hawaii_HI_199.tsv', 'r') as f:
    lines = f.readlines()


t0 = lines[199].strip().split('\t')[0]
t1 = lines[199].strip().split('\t')[1]
print (t0.strip())
print (t1)
print( len(t0.split()), len(t1.split()))
for line in lines:
    ll = line.split('\t')
    tweet = ll[0]
    tweet_pos = ll[1]
    
    cleanTweet(tweet, tweet_pos)



Never give in in nothing , great or small , large or pettyexcept to convictions of honor and good sense . - Winston Churchill #quotes #quote
R V P P N , A & A , A & N P N P N & A N , , ^ ^ # #
26 26
